In [39]:
import gurobipy as gp

from gurobipy import GRB
from scipy.stats import norm

EPS = 10**-4

model = gp.Model()
model.setParam("OutputFlag", 0)
model.setParam("NonConvex", 2)

rho_1 = model.addVar(name='rho_1', lb = -10**4, ub = 10**4, vtype=GRB.CONTINUOUS)
rho_2 = model.addVar(name='rho_2', lb = -10**4, ub = 10**4, vtype=GRB.CONTINUOUS)
rho = model.addVar(name='rho', lb = -10**4, ub = 10**4, vtype=GRB.CONTINUOUS)
sigma_1 = model.addVar(name='sigma_1', ub = 1, vtype=GRB.CONTINUOUS)
sigma_2 = model.addVar(name='sigma_2', ub = 1, vtype=GRB.CONTINUOUS)
sigma = model.addVar(name='sigma', ub = 1, vtype=GRB.CONTINUOUS)
sigma_squared = model.addVar(name='sigma_squared', ub = 10**4, vtype=GRB.CONTINUOUS)
c = model.addVar(name='c', lb = 1, ub = 3, vtype=GRB.CONTINUOUS)
p = model.addVar(name='p', lb = 0.95, ub = 1, vtype=GRB.CONTINUOUS)

anx_1 = model.addVar(name='anx_1', lb = -10**4, ub = 10**4, vtype=GRB.CONTINUOUS)
anx_2 = model.addVar(name='anx_2', lb = -10**4, ub = 10**4, vtype=GRB.CONTINUOUS)
anx_3 = model.addVar(name='anx_3', lb = -10**4, ub = 10**4, vtype=GRB.CONTINUOUS)
model.update()

model.addConstr(rho_1 == 0.2 - norm.ppf(0.95)*sigma_1)
model.addConstr(rho_2 == c - (norm.ppf(0.9)+(norm.ppf(1-10**-16)-norm.ppf(0.9))/(1-10**-16-0.9)*(p-0.9))*sigma_2)
model.addConstr(sigma_squared == sigma_1*sigma_1 + sigma_2*sigma_2)
model.addGenConstrPow(sigma_squared, sigma, 0.5)
model.addConstr(rho == 0.2 - norm.ppf(0.99)*sigma)
model.addConstr(anx_1 == gp.min_(rho_1, rho_2))
model.addConstr(anx_2 == -anx_1)
model.addConstr(anx_3 == gp.max_(anx_2, rho))
model.addConstr(anx_3 <= -EPS)
# model.addConstr(anx_3 >= 0)
model.update()

model.write('MILP.lp')
model.optimize()

if model.getAttr("Status") == 2:
    for v in model.getVars():
        print('%s %g' % (v.varName, v.x))

rho_1 0.2
rho_2 0.25742
rho -0.0104261
sigma_1 0
sigma_2 0.0904534
sigma 0.0904534
sigma_squared 0.00818182
c 1
p 1
anx_1 0.2
anx_2 -0.2
anx_3 -0.0104261
